## 处理步骤

- 注册 https://developer.nytimes.com/ 开发者
- 利用 Article Search API 获取文章信息 https://developer.nytimes.com/docs/articlesearch-product/1/overview
  - 支持 begin_date, end_date, q(搜索词), page(翻页,单次只能返回10条结果)
  - 返回的结果为 json 类型, 包括 Article ID, Artcile URL, Headline, Keyword, Authors 等信息
- 通过关键字 Covid-19, coronavirus, vaccine 分别调用 nyt_api, 取回结果, python 代码为 nytimes_search_api.py
- 对 api 返回的三个关键字的结果, 利用 Article ID 去重, Covid-19(1667), coronavirus(1849), vaccine(362), 最后得到 1956 篇文章, 保存为 articles_nytimes_opinion.csv 和 articles_nytimes_opinion.meta, meta是json格式, 方便后续使用. python 代码为 parse_api_data.py
- 利用整理好的 Article 信息, 抓取 html 页面整体文本并保存, 方便后续解析, 每篇文章一个html文件, 文件名用 Article ID 命名. python 代码为 get_html.py
- 依次解析html文本, 利用 xpath 解析html中文章主体部分, 解析时注意保留文章分段信息, 去掉 nytimes 末尾信息(与内容无关), 并利用 nltk 对 Tokenizer 做了处理, 生成语料库的文本文件, 文件名为 [pub_date] [no] [headline] [authers].txt, 并统计corpus的word_count, 保存到 word_count.csv中, python 代码为 parse_html.py
- 解析中发现, 有文本词数为0的, 确认后发现为全图片文章, 过滤掉, 最终文章数 1873, words 数量 188w+, 整理文章meta信息并保存到 corpus_meta.csv 中

## Data Summary

In [21]:
library(tidyverse)
library(knitr)
library(skimr)

In [5]:
articles <- read_csv(file = "articles_nytimes_opinion.csv")

Parsed with column specification:
cols(
  nyt_id = col_character(),
  doc_id = col_character(),
  doc_type = col_character(),
  word_count = col_double(),
  pub_date = col_date(format = ""),
  headline = col_character(),
  abstract = col_character(),
  subsection_name = col_character(),
  snippet = col_character(),
  lead_paragraph = col_character(),
  source = col_character(),
  authors = col_character(),
  web_url = col_character(),
  keywords = col_character()
)



In [6]:
articles %>%
    skim()

── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             1956      
Number of columns          14        
_______________________              
Column type frequency:               
  character                12        
  Date                     1         
  numeric                  1         
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
   skim_variable   n_missing complete_rate   min   max empty n_unique whitespace
 1 nyt_id                  0         1        48    54     0     1956          0
 2 doc_id                  0         1        36    36     0     1956          0
 3 doc_type                0         1         5    11     0        3          0
 4 headline                0         1         6    86     0     1929          0
 5 abstract                1         0.999     3   220

In [7]:
articles %>%
    summarise(sum_word_count = sum(word_count))

sum_word_count
<dbl>
1976299


In [8]:
articles %>% head()

nyt_id,doc_id,doc_type,word_count,pub_date,headline,abstract,subsection_name,snippet,lead_paragraph,source,authors,web_url,keywords
<chr>,<chr>,<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
nyt://article/09a65af8-ec78-5da7-99a0-b5eabd4766c0,09a65af8-ec78-5da7-99a0-b5eabd4766c0,article,913,2020-07-31,Let’s End the Wait for Coronavirus Test Results. Here’s How.,States should look to New York’s strategies.,NA,States should look to New York’s strategies.,"It’s been six months since the United States reported its first coronavirus case, and getting a test can still take days. National labs are overwhelmed, leaving people to wait as much as two weeks for results. Every day that testing falls short is another day the virus can spread undetected, costing lives and delaying the reopening of our economy, schools and society.",The New York Times,Andrew None,https://www.nytimes.com/2020/07/31/opinion/coronavirus-testing-cuomo.html,"subject: Tests (Medical),subject: Coronavirus (2019-nCoV),subject: Laboratories and Scientific Equipment,glocations: New York State"
nyt://article/e1752a66-dfbd-5b58-a6c4-44dcc2670416,e1752a66-dfbd-5b58-a6c4-44dcc2670416,article,588,2020-07-31,Why the Virus Is Getting the Better of Us,Readers outline the mistakes made that have caused new cases to soar and suggest better approaches.,Letters,Readers outline the mistakes made that have caused new cases to soar and suggest better approaches.,To the Editor:,The New York Times,NA,https://www.nytimes.com/2020/07/31/opinion/letters/coronavirus-economy.html,"subject: Coronavirus (2019-nCoV),glocations: United States"
nyt://article/b9ae229c-3f2e-56b7-a5b6-e11a7ebb035f,b9ae229c-3f2e-56b7-a5b6-e11a7ebb035f,article,613,2020-07-31,So Trump Wants to Postpone the Election,One reader views the floating of the idea as “a measure of how desperate he is to stay in power.” Also: Maskless shoppers; uncertain in Australia.,Letters,One reader views the floating of the idea as “a measure of how desperate he is to stay in power.” Also: Maskless shoppers; uncertain in Australia.,To the Editor:,The New York Times,NA,https://www.nytimes.com/2020/07/31/opinion/letters/trump-election.html,"subject: Presidential Election of 2020,persons: Trump, Donald J,subject: Masks,subject: Shopping and Retail,subject: Coronavirus (2019-nCoV),glocations: Australia"
nyt://article/c9297da4-dce8-5712-adc6-2900e35eadbd,c9297da4-dce8-5712-adc6-2900e35eadbd,article,1265,2020-07-31,Scared That Covid-19 Immunity Won’t Last? Don’t Be,"Dropping antibody counts aren’t a sign that our immune system is failing against the coronavirus, nor an omen that we can’t develop a viable vaccine.",NA,"Dropping antibody counts aren’t a sign that our immune system is failing against the coronavirus, nor an omen that we can’t develop a viable vaccine.","Within the last couple of months, several scientific studies have come out — some peer-reviewed, others not — indicating that the antibody response of people infected with SARS-CoV-2 dropped significantly within two months. The news has sparked fears that the very immunity of patients with Covid-19 may be waning fast — dampening hopes for the development of an effective and durable vaccine.",The New York Times,"Akiko Iwasaki, Ruslan Medzhitov",https://www.nytimes.com/2020/07/31/opinion/coronavirus-antibodies-immunity.html,"subject: Disease Rates,subject: Coronavirus (2019-nCoV),subject: Vaccination and Immunization,subject: Immune System,subject: Antibodies,subject: Viruses"
nyt://article/61a75364-591a-5822-84be-21c6504b8e07,61a75364-591a-5822-84be-21c6504b8e07,article,4093,2020-07-31,"A Friendship, a Pandemic and a Death Beside the Highway",How a photograph of a young man cradling his dying friend sent me on a journey across India.,Sunday Review,How a photograph of a young man cradling his dying friend sent me on a journey across India.,"DEVARI, India — Somebody took a photograph on the side of a highway in India.",The New York Times,Basharat Peer,https:/

In [9]:
corpus <- read_csv("word_count.csv")

Parsed with column specification:
cols(
  doc_id = col_character(),
  word_count = col_double(),
  idx = col_double(),
  src_html = col_character()
)



In [23]:
corpus %>% head()
corpus %>% skim()

doc_id,word_count,idx,src_html
<chr>,<dbl>,<dbl>,<chr>
09a65af8-ec78-5da7-99a0-b5eabd4766c0,841,1,data_html/09a65af8-ec78-5da7-99a0-b5eabd4766c0.html
e1752a66-dfbd-5b58-a6c4-44dcc2670416,579,2,data_html/e1752a66-dfbd-5b58-a6c4-44dcc2670416.html
b9ae229c-3f2e-56b7-a5b6-e11a7ebb035f,606,3,data_html/b9ae229c-3f2e-56b7-a5b6-e11a7ebb035f.html
c9297da4-dce8-5712-adc6-2900e35eadbd,1205,4,data_html/c9297da4-dce8-5712-adc6-2900e35eadbd.html
61a75364-591a-5822-84be-21c6504b8e07,3875,5,data_html/61a75364-591a-5822-84be-21c6504b8e07.html
312ca6c1-1c4d-5b06-af0b-6bbbe9b341d9,777,6,data_html/312ca6c1-1c4d-5b06-af0b-6bbbe9b341d9.html


── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             1873      
Number of columns          4         
_______________________              
Column type frequency:               
  character                2         
  numeric                  2         
________________________             
Group variables            None      

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate   min   max empty n_unique whitespace
1 doc_id                0             1    36    36     0     1873          0
2 src_html              0             1    51    51     0     1873          0

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean    sd    p0   p25   p50   p75
1 word_count            0             1 1004.  445.    14   790   941  1172
2 idx                   0    

In [25]:
articles %>%
    filter(doc_type == 'article') %>%
    inner_join(corpus, by="doc_id") %>%
    mutate(word_count_diff = word_count.x - word_count.y) %>%
    select_all() %>%
    arrange(desc(word_count_diff)) -> join_df

join_df %>%
    summarise(
        sum_nyt_word_count = sum(word_count.x),
        sum_crp_word_count = sum(word_count.y)
    )

join_df %>%
    rename(
        nyt_word_count=word_count.x,
        crp_word_count=word_count.y)

sum_nyt_word_count,sum_crp_word_count
<dbl>,<dbl>
1975548,1881072


nyt_id,doc_id,doc_type,nyt_word_count,pub_date,headline,abstract,subsection_name,snippet,lead_paragraph,source,authors,web_url,keywords,crp_word_count,idx,src_html,word_count_diff
<chr>,<chr>,<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
nyt://article/61a75364-591a-5822-84be-21c6504b8e07,61a75364-591a-5822-84be-21c6504b8e07,article,4093,2020-07-31,"A Friendship, a Pandemic and a Death Beside the Highway",How a photograph of a young man cradling his dying friend sent me on a journey across India.,Sunday Review,How a photograph of a young man cradling his dying friend sent me on a journey across India.,"DEVARI, India — Somebody took a photograph on the side of a highway in India.",The New York Times,Basharat Peer,https://www.nytimes.com/2020/07/31/opinion/sunday/India-migration-coronavirus.html,"subject: Caste Systems,subject: Economic Conditions and Trends,subject: Coronavirus (2019-nCoV),subject: Labor and Jobs,subject: Remittances,subject: Migrant Labor (Non-Agriculture),subject: Poverty,persons: Modi, Narendra,glocations: India",3875,5,data_html/61a75364-591a-5822-84be-21c6504b8e07.html,218
nyt://article/dc695bd2-3567-5bd5-97aa-59c89c813f98,dc695bd2-3567-5bd5-97aa-59c89c813f98,article,3498,2020-06-26,‘Let Freedom Ring’ From Georgia,"“The fact I am here means I am descended from people who, even enslaved, did not give up hope.”",NA,"“The fact I am here means I am descended from people who, even enslaved, did not give up hope.”","ATLANTA — This, an old saying goes, is “the city too busy to hate,” one of the few places in America where enlightened leaders, black and white, chose prosperity over prejudice and a large black middle class emerged decades ago. Birthplace of Martin Luther King Jr., cradle of the civil rights movement, Atlanta, with its gleaming towers and porch swings, was an American exception. The city managed racial conflict through compromise. It was the black mecca. Or so the story went.",The New York Times,Roger Cohen,https://www.nytimes.com/2020/06/26/opinion/let-freedom-ring-from-georgia.html,"subject: Black Lives Matter Movement,subject: George Floyd Protests (2020),glocations: Atlanta (Ga),glocations: Georgia,subject: Police Brutality, Misconduct and Shootings,persons: Arbery, Ahmaud (1994-2020),persons: Brooks, Rayshard (d 2020),persons: Trump, Donald J,persons: Abrams, Stacey Y,persons: Biden, Joseph R Jr,persons: Floyd, George (d 2020)",3284,229,data_html/dc695bd2-3567-5bd5-97aa-59c89c813f98.html,214
nyt://article/641d2506-92a0-53a0-89d4-6bb0329b40fc,641d2506-92a0-53a0-89d4-6bb0329b40fc,article,3542,2020-04-03,"Of the Virus and God, Orange Peels and the Party","A mother and daughter write through distance and disease, searching for the other’s love.",Sunday Review,"A mother and daughter write through distance and disease, searching for the other’s love.",CHICAGO — My mother believes that God and the Chinese Communist Party will defeat the novel coronavirus.,The New York Times,Yangyang Cheng,https://www.nytimes.com/2020/04/03/opinion/sunday/coronavirus-china-US.html,"subject: Coronavirus (2019-nCoV),subject: Politics and Government,subject: Quarantines,subject: Censorship,subject: Stockpiling,subject: Epidemics,subject: Deaths (Fatalities),subject: Protective Clothing and Gear,subject: News and News Media,subject: Masks,subject: Shortages,organizations: Communist Party of China,persons: Xi Jinping,glocations: Chicago (Ill),glocations: China,glocations: United States,glocations: Wuhan (China)",3350,1259,data_html/641d2506-92a0-53a0-89d4-6bb0329b40fc.html,192
nyt://article/b34d3448-c5bb-5433-9e70-86c057c0542e,b34d3448-c5bb-5433-9e70-86c057c0542e,article,1677,2020-05-03,Flattening the Truth on Coronavirus,"All your questions about the pandemic, answered. Sort of.",NA,"All your questions about the pandemic, answered. Sort of.",The People (P): What is happening?,The New York Times,Dave Eggers,https://www.nytimes.com/2020/05/03/opinion/dave-eggers-coronavirus-q

In [15]:
join_df %>% write_csv("corpus_meta.csv")

- The NYT has articles that are not in the corpus

In [19]:
articles %>% 
    anti_join(corpus, by="doc_id") %>%
    arrange(desc(word_count))

nyt_id,doc_id,doc_type,word_count,pub_date,headline,abstract,subsection_name,snippet,lead_paragraph,source,authors,web_url,keywords
<chr>,<chr>,<chr>,<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
nyt://article/a9e9f06d-f3a9-51fb-b6fc-d8ebc46fdba2,a9e9f06d-f3a9-51fb-b6fc-d8ebc46fdba2,article,741,2020-07-02,Your Fourth of July Quiz,Let’s see how well you’ve been following politics.,NA,Let’s see how well you’ve been following politics.,"Happy Independence Day! You definitely deserve a holiday, people. It’s tough trying to keep up on current events when you want to block out four-fifths of what’s going on.",The New York Times,Gail Collins,https://www.nytimes.com/2020/07/02/opinion/your-fourth-of-july-quiz.html,"subject: United States Politics and Government,subject: Presidential Election of 2020,subject: Impeachment,persons: Biden, Joseph R Jr,persons: Trump, Donald J Jr,persons: DeSantis, Ron,persons: Collins, Susan M,persons: Gaetz, Matt,persons: Trump, Eric F (1984- ),persons: Trump, Donald J"
nyt://article/00fa85c4-07dd-51a4-9fbd-5b3cd125b725,00fa85c4-07dd-51a4-9fbd-5b3cd125b725,article,10,2020-06-16,Op-Art: A Surviving Instrument,A luthier under lockdown finds resilience in the history of his craft.,NA,A luthier under lockdown finds resilience in the history of his craft.,Julia Rothman is an illustrator. Jacob Curtz is a luthier.,The New York Times,"Julia Rothman, Jacob Curtz",https://www.nytimes.com/2020/06/16/opinion/violin-luthier-coronavirus.html,"subject: Violins and Violas,subject: Coronavirus (2019-nCoV),subject: Music,subject: Classical Music,subject: Wood and Wood Products,subject: Handicrafts"
nyt://video/4e0d4294-a927-5321-92f4-270614f8ee15,4e0d4294-a927-5321-92f4-270614f8ee15,video,0,2020-07-27,‘That’s Ridiculous.’ How America’s Coronavirus Response Looks Abroad.,"From lockdowns to testing, we showed people from around the world the facts and figures on how America handled the virus. It wasn’t pretty.",NA,"From lockdowns to testing, we showed people from around the world the facts and figures on how America handled the virus. It wasn’t pretty.","From lockdowns to testing, we showed people from around the world the facts and figures on how America handled the virus. It wasn’t pretty.",The New York Times,"Brendan Miller, Adam Westbrook",https://www.nytimes.com/video/opinion/100000007227777/covid-19-global-response.html,"subject: Coronavirus (2019-nCoV),persons: Trump, Donald J,subject: Disease Rates,subject: Quarantines,subject: Unemployment,subject: Tests (Medical),subject: Deaths (Fatalities),subject: Stimulus (Economic),glocations: United States,subject: United States Politics and Government,subject: your-feed-opinionvideo,glocations: Singapore,glocations: Jordan,glocations: Great Britain,glocations: New Zealand,glocations: Senegal,glocations: South Korea,glocations: Germany"
nyt://article/f19e8549-0854-519e-a1ef-2b0b7ef1d95e,f19e8549-0854-519e-a1ef-2b0b7ef1d95e,article,0,2020-07-20,I Am Stuck Between Two Lives During This Pandemic,An artist battles anxiety by trying to stay present.,NA,An artist battles anxiety by trying to stay present.,NA,The New York Times,Lucie Langston,https://www.nytimes.com/2020/07/20/opinion/coronavirus-anxiety.html,"subject: Comic Books and Strips,subject: Coronavirus (2019-nCoV),subject: Anxiety and Stress,subject: Quarantine (Life and Culture),subject: Art"
nyt://video/6270116f-6e3e-540a-8aa9-d2e946b12555,6270116f-6e3e-540a-8aa9-d2e946b12555,video,0,2020-07-19,"Clearly, the Virus Isn’t the Only Problem",There’s a reason the U.S. is failing at containing Covid-19.,NA,There’s a reason the U.S. is failing at containing Covid-19.,There’s a reason the U.S. is failing at containing Covid-19.,The New York Times,"Chai Dingari, Asaf Kastner",https://www.nytimes.com/video/opinion/100000007242791/coronavirus-response-america.html,"subject: Coronavirus (2019-nCoV),subject: Coronavirus Reopenings,persons: Trump, Donald J,subject: your-feed-opinionvideo,glocations: United State